In [5]:
import numpy as np
import glob
import cv2
import sklearn
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from tensorflow.python import keras
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from random_eraser import get_random_eraser
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Dense, GlobalMaxPooling2D, Flatten, Dropout, GaussianNoise, Activation
import os, random
import shutil

In [25]:
## loading data
folder = r'M:\Projects\number plates\hsrp_plates\*'
hsrp = np.zeros((1482,12288))
nhsrp = np.zeros((1061,12288))
i=0
for path in glob.glob(folder):
    im = cv2.imread(path)
    im1 = cv2.resize(im,(64,64)).flatten()
    hsrp[i,:]=im1
    i=i+1
folders = r'M:\Projects\number plates\non_hsrp_plates\*'
j=0
for path in glob.glob(folders):
    img = cv2.imread(path)
    img1 = cv2.resize(img,(64,64)).flatten()
    nhsrp[j,:]=img1
    j=j+1

In [32]:
hsrp_y = np.zeros((1482))
nhsrp_y = np.ones((1061))
train_x = np.concatenate((hsrp[:1200],nhsrp[:900])) 
train_y = np.concatenate((hsrp_y[:1200],nhsrp_y[:900])) 
val_x = np.concatenate((hsrp[1200:],nhsrp[900:])) 
val_y = np.concatenate((hsrp_y[1200:],nhsrp_y[900:]))
print(train_x.shape,train_y.shape)
print(val_x.shape,val_y.shape)


(2100, 12288) (2100,)
(443, 12288) (443,)


In [46]:
clf = LogisticRegression(penalty='l2', max_iter=70,verbose=1,random_state=2,C=10,solver='newton-cg').fit(train_x, train_y)
clf.score(val_x, val_y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   14.9s finished


0.8261851015801355

In [27]:
train_generator = ImageDataGenerator(rescale=1/255,
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    brightness_range=(0.1,0.9),
                                    channel_shift_range = 150,
                                    fill_mode="nearest",
                                    preprocessing_function = get_random_eraser(v_l=0, v_h=1))
                            
train = train_generator.flow_from_directory(r'M:\Projects\number plates\train',target_size=(224,224),class_mode='categorical',shuffle=True)

Found 1995 images belonging to 2 classes.


In [28]:
val_generator = ImageDataGenerator(rescale=1./255)
val = val_generator.flow_from_directory(r'M:\Projects\number plates\val',target_size=(224,224),class_mode='categorical',shuffle=False)

Found 450 images belonging to 2 classes.


In [34]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>=0.80):
            if(logs.get('val_acc')>=0.75):
                   print("\nReached 75.0% accuracy so cancelling training!")
                   self.model.stop_training = True

In [35]:
base_model = VGG16(weights='imagenet',include_top=False)

In [36]:
for layer in base_model.layers:
    layer.trainable = False

In [37]:
x = base_model.output
x = GlobalMaxPooling2D()(x)
x = Dense(1000,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(50,activation='relu')(x)
x = Dropout(0.5)(x)
pred = Dense(2,activation='softmax')(x)
model = Model(inputs = base_model.input, outputs = pred)

In [38]:
callbacks= myCallback()
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
history = model.fit_generator(train,epochs=30,callbacks=[callbacks],validation_data = val)

Epoch 1/30
63/63 [==============================] - 1036s 16s/step - loss: 0.7387 - acc: 0.5594 - val_loss: 0.6605 - val_acc: 0.6667
Epoch 2/30
63/63 [==============================] - 998s 16s/step - loss: 0.6854 - acc: 0.5779 - val_loss: 0.6851 - val_acc: 0.6733
Epoch 3/30
63/63 [==============================] - 967s 15s/step - loss: 0.6831 - acc: 0.5674 - val_loss: 0.6724 - val_acc: 0.7156
Epoch 4/30
63/63 [==============================] - 941s 15s/step - loss: 0.6742 - acc: 0.5825 - val_loss: 0.6585 - val_acc: 0.6667
Epoch 5/30
63/63 [==============================] - 958s 15s/step - loss: 0.6649 - acc: 0.5960 - val_loss: 0.6376 - val_acc: 0.7156
Epoch 6/30
63/63 [==============================] - 958s 15s/step - loss: 0.6486 - acc: 0.6291 - val_loss: 0.6270 - val_acc: 0.6689
Epoch 7/30
63/63 [==============================] - 939s 15s/step - loss: 0.6334 - acc: 0.6451 - val_loss: 0.5944 - val_acc: 0.7200
Epoch 8/30
63/63 [==============================] - 890s 14s/step - loss: 0

In [1]:
model.save('72random')

NameError: name 'model' is not defined